# Notebook 5: Policy Iteration\n\n**Author:** Divyansh Atri\n\n## Overview\n\nPolicy iteration is an alternative to value iteration for solving HJB equations.\n\n**Algorithm:**\n1. Policy evaluation: Solve linear PDE for fixed policy\n2. Policy improvement: Update policy via Hamiltonian minimization\n3. Repeat until convergence

In [ ]:
import numpy as np\nimport matplotlib.pyplot as plt\nimport sys\nsys.path.append('..')\nfrom utils import *\n\nplt.style.use('seaborn-v0_8-darkgrid')\nplt.rcParams['figure.figsize'] = (14, 6)\n\nprint('Policy Iteration - Ready')

## 1. Policy Iteration Algorithm\n\n**Input:** Initial policy $u^0(t,x)$\n\n**Iterate:**\n1. **Policy Evaluation:** Solve for $V^k$ given $u^k$:\n   $$V_t + L(x, u^k) + b(x, u^k) V_x + \frac{1}{2}\sigma^2 V_{xx} = 0$$\n\n2. **Policy Improvement:** Update policy:\n   $$u^{k+1}(t,x) = \arg\min_u \left\{ L(x,u) + b(x,u) V_x^k \right\}$$\n\n**Stop:** When $\|u^{k+1} - u^k\| < \epsilon$

In [ ]:
# Setup problem\nmodel = MeanRevertingModel(theta=2.0, mu=0.0, sigma=0.5)\ncost_fn = QuadraticCost(q=1.0, r=1.0, q_terminal=5.0)\n\nx_min, x_max, nx = -2.0, 2.0, 81\nT, nt = 1.0, 101\n\nprint(f'Mean-Reverting Model: θ={model.theta}, μ={model.mu}, σ={model.sigma}')\nprint(f'Grid: x ∈ [{x_min}, {x_max}], {nx} points')\nprint(f'Time: t ∈ [0, {T}], {nt} points')

## 2. Policy Iteration Solver

In [ ]:
# Create solver\npi_solver = PolicyIterationSolver(x_min, x_max, nx, T, nt, model, cost_fn)\n\nprint('Running policy iteration...')\nV_pi, u_pi = pi_solver.solve(u_bounds=(-5, 5), max_iter=20, tol=1e-4, verbose=True)\n\nprint(f'\nPolicy iteration converged')\nprint(f'V(0, 0) = {V_pi[0, nx//2]:.6f}')

## 3. Comparison with Value Iteration

In [ ]:
# Solve same problem with value iteration (standard HJB solver)\nvi_solver = HJBSolver(x_min, x_max, nx, T, nt, model, cost_fn)\n\nprint('Running value iteration...')\nV_vi, u_vi = vi_solver.solve_backward(u_bounds=(-5, 5), verbose=False)\n\nprint(f'Value iteration complete')\nprint(f'V(0, 0) = {V_vi[0, nx//2]:.6f}')\n\n# Compare\nV_diff = np.abs(V_pi - V_vi)\nu_diff = np.abs(u_pi - u_vi)\n\nprint(f'\nDifference between methods:')\nprint(f'  Max value difference: {np.max(V_diff):.6e}')\nprint(f'  Max control difference: {np.max(u_diff):.6e}')

## 4. Visualization

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(14, 10))\n\n# Value functions\nx = pi_solver.x\naxes[0,0].plot(x, V_pi[0, :], 'b-', linewidth=2, label='Policy Iteration')\naxes[0,0].plot(x, V_vi[0, :], 'r--', linewidth=2, label='Value Iteration')\naxes[0,0].set_xlabel('State $x$')\naxes[0,0].set_ylabel('Value $V(0,x)$')\naxes[0,0].set_title('Value Function at $t=0$')\naxes[0,0].legend()\naxes[0,0].grid(True, alpha=0.3)\n\n# Optimal controls\naxes[0,1].plot(x, u_pi[0, :], 'b-', linewidth=2, label='Policy Iteration')\naxes[0,1].plot(x, u_vi[0, :], 'r--', linewidth=2, label='Value Iteration')\naxes[0,1].set_xlabel('State $x$')\naxes[0,1].set_ylabel('Control $u^*(0,x)$')\naxes[0,1].set_title('Optimal Control at $t=0$')\naxes[0,1].legend()\naxes[0,1].grid(True, alpha=0.3)\naxes[0,1].axhline(0, color='k', linestyle='--', alpha=0.5)\n\n# Difference in value\nt = pi_solver.t\nT_grid, X_grid = np.meshgrid(t, x, indexing='ij')\nim1 = axes[1,0].contourf(T_grid, X_grid, V_diff, levels=20, cmap='hot')\naxes[1,0].set_xlabel('Time $t$')\naxes[1,0].set_ylabel('State $x$')\naxes[1,0].set_title('Value Difference $|V_{PI} - V_{VI}|$')\nplt.colorbar(im1, ax=axes[1,0])\n\n# Difference in control\nim2 = axes[1,1].contourf(T_grid, X_grid, u_diff, levels=20, cmap='hot')\naxes[1,1].set_xlabel('Time $t$')\naxes[1,1].set_ylabel('State $x$')\naxes[1,1].set_title('Control Difference $|u_{PI}^* - u_{VI}^*|$')\nplt.colorbar(im2, ax=axes[1,1])\n\nplt.tight_layout()\nplt.savefig('../plots/policy_iteration.png', dpi=150, bbox_inches='tight')\nplt.show()

## Summary\n\n**Policy Iteration:**\n- Alternates between policy evaluation and improvement\n- Often converges in fewer iterations than value iteration\n- Each iteration solves a linear PDE (more expensive per iteration)\n\n**Comparison:**\n- Both methods converge to the same solution\n- Choice depends on problem structure and computational resources\n\n**Next:** Simulation-based verification.